In [9]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import classification_report
import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import WeightedRandomSampler

# GPU 디버깅을 위한 환경 변수 설정
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"



In [10]:
# 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.image_paths = []
        self.labels = []

        normal_dir = os.path.join(data_dir, 'NORMAL')
        for img_name in os.listdir(normal_dir):
            self.image_paths.append(os.path.join(normal_dir, img_name))
            self.labels.append(0)

        pneumonia_dir = os.path.join(data_dir, 'PNEUMONIA')
        for img_name in os.listdir(pneumonia_dir):
            self.image_paths.append(os.path.join(pneumonia_dir, img_name))
            self.labels.append(1)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('L')
        label = self.labels[idx]
        return np.array(image), label
    
# transform을 적용한 새로운 데이터셋 클래스 정의
class TransformedDataset(Dataset):
    def __init__(self, dataset, indices, transform=None):
        self.dataset = Subset(dataset, indices)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image, label


In [11]:

# 데이터 증강 변환 추가
train_transform = A.Compose([
    A.Resize(224, 224),
    A.Rotate(limit=5),  # 5도 이내로만 회전
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=5, p=0.5),  # 회전 범위 제한
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.2),
    A.ElasticTransform(p=0.3),
    A.GridDistortion(p=0.3),
    A.CLAHE(p=0.3),
    A.RandomResizedCrop(224, 224, scale=(0.8, 1.0), ratio=(0.75, 1.33), p=0.5),
    A.Normalize(mean=(0.5,), std=(0.5,)),
    ToTensorV2()
])


val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.5,), std=(0.5,)),
    ToTensorV2()
])



In [12]:
# 데이터셋 로드 및 전처리
full_dataset = CustomDataset(data_dir='chest_xray/train')
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_indices, val_indices = random_split(full_dataset, [train_size, val_size])

train_dataset = TransformedDataset(full_dataset, train_indices.indices, transform=train_transform)
val_dataset = TransformedDataset(full_dataset, val_indices.indices, transform=val_transform)


# 데이터셋 로드 후 클래스 불균형 계산
normal_count = len([label for label in full_dataset.labels if label == 0])
pneumonia_count = len([label for label in full_dataset.labels if label == 1])

# 클래스 불균형을 고려한 샘플링 가중치 계산
class_weights = 1. / torch.tensor([normal_count, pneumonia_count], dtype=torch.float)
sample_weights = [class_weights[label] for label in full_dataset.labels]
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# DataLoader에 WeightedRandomSampler 추가
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


In [13]:
class ImprovedCNN(nn.Module):
    def __init__(self):
        super(ImprovedCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # Dropout 추가
        self.dropout = nn.Dropout(0.5)
        
        # Fully connected layers
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = x.view(-1, 256 * 14 * 14)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x



In [15]:
# 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ImprovedCNN().to(device)

# 옵티마이저와 스케줄러 설정
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  # L2 정규화 추가
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

# 데이터셋 로드 후 클래스 불균형 계산
normal_count = len([label for label in full_dataset.labels if label == 0])
pneumonia_count = len([label for label in full_dataset.labels if label == 1])
class_weights = torch.tensor([1.0, normal_count / pneumonia_count], device=device)

# 손실 함수 정의 (가중치 적용)
criterion = nn.CrossEntropyLoss(weight=class_weights)

num_epochs = 30
patience = 7
best_val_loss = float('inf')
early_stop_counter = 0


In [16]:

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    all_train_labels = []
    all_train_predictions = []
    correct = 0
    total = 0

    for images, labels in tqdm.tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        _, predicted = torch.max(outputs.data, 1)
        all_train_labels.extend(labels.cpu().numpy())
        all_train_predictions.extend(predicted.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    train_accuracy = 100 * correct / total
    train_report = classification_report(all_train_labels, all_train_predictions, target_names=['NORMAL', 'PNEUMONIA'], output_dict=True)
    train_recall = train_report['weighted avg']['recall']
    print(f'Epoch [{epoch+1}/{num_epochs}] - Loss: {running_loss / len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%, recall: {train_recall:.4f}')

    # Validation step
    model.eval()
    val_loss = 0.0
    all_val_labels = []
    all_val_predictions = []
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            all_val_labels.extend(labels.cpu().numpy())
            all_val_predictions.extend(predicted.cpu().numpy())
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    val_report = classification_report(all_val_labels, all_val_predictions, target_names=['NORMAL', 'PNEUMONIA'], output_dict=True)
    val_recall = val_report['weighted avg']['recall']
    print(f'Validation Loss: {val_loss / len(val_loader):.4f}, Valid Accuracy: {val_accuracy:.2f}%, recall: {val_recall:.4f}')

    # 학습률 스케줄러 스텝
    scheduler.step(val_loss)

    # 조기 종료(Early Stopping) 체크
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break


100%|██████████| 131/131 [04:58<00:00,  2.28s/it]

Epoch [1/30] - Loss: 1.2952, Train Accuracy: 85.28%, recall: 0.8528


Validation Loss: 0.1306, Valid Accuracy: 95.50%, recall: 0.9550


100%|██████████| 131/131 [04:31<00:00,  2.07s/it]

Epoch [2/30] - Loss: 0.2002, Train Accuracy: 91.87%, recall: 0.9187


...

100%|██████████| 131/131 [04:49<00:00,  2.21s/it]

Epoch [8/30] - Loss: 0.1461, Train Accuracy: 94.32%, recall: 0.9432


Validation Loss: 0.0651, Valid Accuracy: 97.51%, recall: 0.9751


 47%|████▋     | 62/131 [02:13<02:28,  2.15s/it]


KeyboardInterrupt: 

In [ ]:

# 테스트 데이터셋 준비 및 평가
test_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.5,), std=(0.5,)),
    ToTensorV2()
])
test_dataset = CustomDataset(data_dir='chest_xray/test')
test_dataset = TransformedDataset(test_dataset, range(len(test_dataset)), transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

model.eval()
all_labels = []
all_predictions = []
total = 0
correct = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

report = classification_report(all_labels, all_predictions, target_names=['NORMAL', 'PNEUMONIA'], output_dict=True)
weighted_avg_recall = report['weighted avg']['recall']
print("Classification Report:")
print(classification_report(all_labels, all_predictions, target_names=['NORMAL', 'PNEUMONIA']))
print(f"Weighted Average Recall: {weighted_avg_recall:.4f}")
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.4f}%')


Classification Report:
              precision    recall  f1-score   support

      NORMAL       0.96      0.46      0.62       242
   PNEUMONIA       0.75      0.99      0.85       398

    accuracy                           0.79       640
   macro avg       0.85      0.72      0.74       640
weighted avg       0.83      0.79      0.76       640

Weighted Average Recall: 0.7875
Accuracy: 78.7500%
